# LNS Example 1: Knapsack (Version for printing a simple example)

The first example is the knapsack problem. We have a set of items $I$ with a weight $w_i$ and a value $v_i$.
We want to select a subset of items such that the total weight does not exceed a given capacity $C$ and the total value is maximized.

$\max \sum_{i \in I} v_i x_i$
$\text{s.t.} \sum_{i \in I} w_i x_i \leq C$
$x_i \in \{0,1\}$

This is one of the simplest NP-hard problems and can be solved with a dynamic programming approach in pseudo-polynomial time.
CP-SAT is also able to solve many large instances of this problem in an instant.
However, its simple structure makes it a good example to demonstrate the use of Large Neighborhood Search, even if the algorithm will
not be of much use for this problem.

In [220]:
# import all dependencies
import typing
import math
import random
from collections import namedtuple

from ortools.sat.python import cp_model

## Instance Generation

First, we need to create some random instances.

In [221]:
# Instance generation
class Item:
  counter = 0
  """
  A simple class to represent an item in the knapsack problem.
  Every instance of this class is unique, i.e., two items with
  the same weight and value are not equal. Otherwise, we could
  only have a single item for each weight and value combination.
  """
  def __init__(self, weight: int, value: int):
    self.weight = weight
    self.value = value
    self._id = Item.counter
    Item.counter += 1
    
  def __repr__(self):
    return f"I_{'{'+str(self._id)+'}'}(w={self.weight}, v={self.value})"
  
  def short_name(self) -> str:
    return f"I_{'{'+str(self._id)+'}'}"

  def __eq__(self, other):
    return id(self) == id(other)

  def __hash__(self):
    return id(self)
  
  def __lt__(self, other):
    return self._id < other._id

class Instance:
  """
  Simple instance container.
  """
  def __init__(self, items: typing.List[Item], capacity: int) -> None:
    self.items = items
    self.capacity = capacity
    assert len(items) > 0
    assert capacity > 0

def random_instance(num_items: int, ratio: float) -> Instance:
  """
  Creates a random instance of the knapsack problem.
  :param num_items: The number of items.
  :param ratio: The ratio between capacity and sum of weights.
  :return: A list of items and a capacity
  """
  items = []
  for i in range(num_items):
    weight = random.randint(10, 20)
    value = round(random.triangular(1, 5, 3)*weight)
    items.append(Item(weight, value))
  capacity = math.ceil(sum(item.weight for item in items) * ratio)
  return Instance(items, capacity)

def value(items: typing.List[Item]) -> int:
  """
  Returns the total value of a list of items.
  """
  return sum(item.value for item in items)


In [222]:
def instance_to_str(instance: Instance, short=False) -> str:
  """
  Prints an instance of the knapsack problem.
  """
  items = ""
  for i, item in enumerate(instance.items):
    if i > 0:
      items += ","
      if not short and i % 5 == 0 and i < len(instance.items) - 1:
        items += "$\n$\quad "
    items += str(item) if not short else item.short_name()
  items = "\\{"+items+"\\}" if short else items
  return "C="+str(instance.capacity)+"$,\n$I="+items

def solution_to_str(solution: typing.List[Item]) -> str:
  """
  Prints a solution to the knapsack problem.
  """
  items = ", ".join(item.short_name() for item in solution)
  items = "\\{"+items+"\\}"
  return items

## Greedy Algorithm

Next, we need an initial solution.
We use a simple greedy algorithm that adds items to the knapsack as long as the capacity is not exceeded.
It would be much smarter to sort the items by value/weight ratio and add the items with the highest ratio first.
However, this would often create near-optimal solution, and then we wouldn't see much improvement from the LNS.

In [223]:
# Simple greedy algorithm for the knapsack problem
def greedy_solution(instance: Instance) -> typing.List[Item]:
  """
  A simple greedy algorithm for the knapsack problem.
  It is bad on purpose, so we can improve it with local search.
  For random instances, the greedy algorithm otherwise often
  finds the (nearly) optimal solution and there is nothing to see.
  """
  solution = []
  remaining_capacity = instance.capacity
  for item in instance.items:
    if item.weight <=  remaining_capacity:
      solution.append(item)
      remaining_capacity -= item.weight
  return solution


## Exact Solver for Subproblem

We will remove items from the knapsack and try to refill it with better items.
This subproblem is the Knapsack problem again, and we can solve it with CP-SAT.

In [224]:
# Exact solver for knapsack

def solve_knapsack(instance: Instance, max_time_in_seconds: float = 90) -> typing.List[Item]:
  """
  Optimal solver for knapsack
  """
  model = cp_model.CpModel()
  x = [model.NewBoolVar(f"x_{i}") for i in range(len(instance.items))]
  model.Add(sum(x[i] * item.weight for i, item in enumerate(instance.items)) <= instance.capacity)
  model.Maximize(sum(x[i] * item.value for i, item  in  enumerate(instance.items)))
  solver = cp_model.CpSolver()
  solver.parameters.max_time_in_seconds = max_time_in_seconds
  #solver.parameters.log_search_progress = True
  status = solver.Solve(model)
  if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    if status == cp_model.FEASIBLE:
      print("Warning: Solver did not find optimal solution. Returned solution is feasible but not optimal.")
    return sorted([item for i, item in enumerate(instance.items) if solver.Value(x[i]) == 1])
  else:
    return []


## Large Neighborhood Search for the Knapsack Problem

### Initialization

We need to start with an initial solution that is then improved by the LNS algorithm.
We use a simple greedy algorithm to generate an initial solution.

In [225]:
# Create instance
instance = random_instance(100, 0.1)
# compute some initial solution
initial_solution = greedy_solution(instance)
print("Instance:", "$"+instance_to_str(instance)+"$")
print()
print(f"Initial solution of value {value(initial_solution)}:", "$"+solution_to_str(initial_solution)+"$")

Instance: $C=151$,
$I=I_{0}(w=12, v=37),I_{1}(w=16, v=49),I_{2}(w=20, v=53),I_{3}(w=11, v=14),I_{4}(w=19, v=42),$
$\quad I_{5}(w=13, v=53),I_{6}(w=18, v=54),I_{7}(w=16, v=56),I_{8}(w=14, v=45),I_{9}(w=12, v=39),$
$\quad I_{10}(w=11, v=42),I_{11}(w=19, v=43),I_{12}(w=12, v=43),I_{13}(w=19, v=66),I_{14}(w=20, v=54),$
$\quad I_{15}(w=13, v=54),I_{16}(w=12, v=33),I_{17}(w=12, v=38),I_{18}(w=14, v=43),I_{19}(w=15, v=28),$
$\quad I_{20}(w=11, v=47),I_{21}(w=10, v=31),I_{22}(w=20, v=97),I_{23}(w=10, v=35),I_{24}(w=19, v=56),$
$\quad I_{25}(w=11, v=33),I_{26}(w=12, v=38),I_{27}(w=15, v=45),I_{28}(w=17, v=58),I_{29}(w=11, v=48),$
$\quad I_{30}(w=15, v=32),I_{31}(w=17, v=67),I_{32}(w=15, v=43),I_{33}(w=16, v=41),I_{34}(w=18, v=42),$
$\quad I_{35}(w=14, v=44),I_{36}(w=20, v=45),I_{37}(w=13, v=50),I_{38}(w=17, v=57),I_{39}(w=17, v=33),$
$\quad I_{40}(w=17, v=49),I_{41}(w=12, v=21),I_{42}(w=14, v=37),I_{43}(w=20, v=74),I_{44}(w=14, v=55),$
$\quad I_{45}(w=10, v=25),I_{46}(w=16, v=26),I_{47}(w=10, v

### Improve the solution via LNS

The LNS algorithm is a heuristic that iteratively destroys and repairs parts of the solution.
We remove a part of the selected item in the current solution and then select some additional
items from the remaining set. Using the exact solver, we find the optimal solution for the
remaining capacity using the selected items. This is repeated for some iterations.

There are two important parameters for the LNS algorithm:
* The size of the subproblem we solve with the exact solver.
* The size of items we remove from the current solution.

In [226]:

class KnapsackLns:
  """
  Knapsack LNS solver.
  """

  def __init__(self, instance: Instance, initial_solution: typing.List[Item], subproblem_size: int):
    self.instance = instance
    self.solution = initial_solution
    self.subproblem_size = subproblem_size  # Number of items to consider in subproblem
    self.solutions = [initial_solution]

  def _remaining_capacity(self):
    return self.instance.capacity - sum(item.weight for item in self.solution)
  
  def _remaining_items(self):
    return list(set(self.instance.items).difference(self.solution))
  
  def _destroy(self, num_items: int) -> typing.List[Item]:
    """
    Destroy a part of the solution by removing num_items from it.
    """
    num_items = min(len(self.solution), num_items)
    assert 0 <= num_items <= self.subproblem_size
    items_removed = random.sample(self.solution,  num_items)
    self.solution = [item for item in self.solution if item not in items_removed]
    print(f"Deleting the following {len(items_removed)} items from the solution: ${solution_to_str(items_removed)}$\n")
    return items_removed
  
  def _repair(self, I_: typing.List[Item], max_time_in_seconds: float = 90):
    """
    Repair the solution by adding items from I_ to it.
    """
    C_ = self._remaining_capacity()
    print("Repairing solution by considering the following subproblem:\n")
    subproblem = Instance(I_, C_)
    print("Subproblem:", "$"+instance_to_str(subproblem, short=True)+"$\n")
    subsolution = solve_knapsack(Instance(I_, C_), max_time_in_seconds)
    print(f"Computed the following solution of value {value(subsolution)} for the subproblem:", "$"+solution_to_str(subsolution)+"$\n")
    print(f"Combining ${solution_to_str(self.solution)}\\cup {solution_to_str(subsolution)}$\n")
    self.solution += subsolution
    self.solution = sorted(self.solution)
    print(f"New solution of value {value(self.solution)}:", "$"+solution_to_str(self.solution)+"$\n")
    assert self._remaining_capacity() >= 0

  def perform_lns_iteration(self, destruction_size: int, max_time_in_seconds: float = 90):
    # 1. Destroy
    assert destruction_size > 0
    items_removed = self._destroy(destruction_size)
    # 2. Build subproblem for repair
    remaining_items = self._remaining_items()
    n = min(self.subproblem_size-destruction_size, len(remaining_items))
    new_items_to_consider = random.sample(remaining_items, n)
    # Add the removed items to the set of items to consider, such that
    # we can also find an equally good solution
    I_ = list(set(items_removed + new_items_to_consider).difference(self.solution))
    # 3. Repair
    self._repair(I_, max_time_in_seconds)
    self.solutions.append(self.solution)


### Run the LNS

Play around with the parameters and see how the LNS algorithm improves the solution.

In [227]:

lns = KnapsackLns(instance, initial_solution, subproblem_size=15)
for i in range(10):
  print(f"**Round ${i+1}$ of LNS algorithm:**\n")
  lns.perform_lns_iteration(destruction_size=5)
  print()
  #print(f"=> Iteration {i}: {value(lns.solution)} (improvement: {value(lns.solution) / value(lns.solutions[0])})")

**Round $1$ of LNS algorithm:**

Deleting the following 5 items from the solution: $\{I_{0}, I_{7}, I_{8}, I_{9}, I_{6}\}$

Repairing solution by considering the following subproblem:

Subproblem: $C=72$,
$I=\{I_{6},I_{9},I_{86},I_{13},I_{47},I_{73},I_{0},I_{8},I_{7},I_{38},I_{57},I_{11},I_{60},I_{14}\}$

Computed the following solution of value 244 for the subproblem: $\{I_{8}, I_{9}, I_{13}, I_{38}, I_{47}\}$

Combining $\{I_{1}, I_{2}, I_{3}, I_{4}, I_{5}\}\cup \{I_{8}, I_{9}, I_{13}, I_{38}, I_{47}\}$

New solution of value 455: $\{I_{1}, I_{2}, I_{3}, I_{4}, I_{5}, I_{8}, I_{9}, I_{13}, I_{38}, I_{47}\}$


**Round $2$ of LNS algorithm:**

Deleting the following 5 items from the solution: $\{I_{3}, I_{13}, I_{2}, I_{9}, I_{1}\}$

Repairing solution by considering the following subproblem:

Subproblem: $C=78$,
$I=\{I_{13},I_{9},I_{84},I_{41},I_{15},I_{42},I_{74},I_{16},I_{3},I_{1},I_{2},I_{67},I_{50},I_{89},I_{43}\}$

Computed the following solution of value 275 for the subproblem: 

## Try to compute an optimal solution

To have a comparison, we can try to compute an optimal solution with the exact solver.

In [228]:
optimal_solution = solve_knapsack(instance, max_time_in_seconds=900)
print(f"CP-SAT solution: {value(optimal_solution)}")

CP-SAT solution: 673


## Conclusion

We are able to improve the initial solution via LNS, however, only because we used a bad greedy algorithm.
If we had used a better greedy algorithm, the LNS algorithm would not be able to improve the solution by much.
However, the LNS algorithm is a very powerful heuristic that can be used to improve solutions for many problems.
This example just had the purpose to demonstrate the implementation of LNS.

## Exercise

Try to generalize the LNS algorithm for Multi-Knapsack problems, where instead of a single knapsack, we have multiple knapsacks with different capacities, and items can have different values and weights for each knapsack.
Multi-Knapsack problems can be significantly harder but also of practical interest for many applications, such as scheduling and resource allocation.